In [ ]:
# Get Data
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# wget on windows
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -OutFile "input.txt"


# ======= DATA PROCESSING  ======= 

[Tutorial Video](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=1050s)

In [4]:
# Install Torch
import torch

file_path = "data/input.txt"
# read it in to inspect it
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()



In [5]:
print("length of dataset in characters: ", len(text))


length of dataset in characters:  1115394


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


# Notes About Encoding and Decoding
1) Encoding is needed to turn input into numbers that can be processed by a NN
2) Decoding is needed to turn encoding output from the NN back into the original format 

In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
print(stoi)
print(itos)
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [30]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    start_index_random = len(data) - block_size
    stop_index_random =  (batch_size,)
    ix = torch.randint(start_index_random, stop_index_random)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
# (1 batch) consists of (4 input samples) blocks and each block has (8 characters) -> (4,8) tensor for x and y
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

# for b in range(batch_size): # batch dimension
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


# ======= TRAINING  =======

In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        #print(f"self.token_embedding_table: {self.token_embedding_table}") 
        #print(f"self.token_embedding_table.weight.shape: {self.token_embedding_table.weight.shape}")


    def forward(self, idx, targets=None):

        # B = batch
        # T = time
        # idx and targets are both (B,T) tensor of integers
        # idx => (4,8) (B,T)
        # Targets => (4,8) (B,T)
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        #rint(f"logits: {logits.shape}")

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #rint(f"logits: {logits.shape}, targets: {targets.shape}")
            loss = F.cross_entropy(logits, targets)
            # print("targets: ")
            # print(targets)
            # print("logits: ")
            # print(logits)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # print(f"idx: {idx.shape}")
            # print(f"logits: {logits.shape}")
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
#print(logits.shape)
#print(loss)

start_test_idx = torch.zeros((1, 1), dtype=torch.long)
encoded_output = m.generate(idx = start_test_idx, max_new_tokens=10)
print(decode(encoded_output[0].tolist()))



Sr?qP-QWkt


In [24]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:

for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.62705659866333


In [32]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


rCxiObOtNKuyiqMqlwfnoC
C f?:BoCpj:pdmBUVtifSPT;Ioq.R33YI&yN:tT;oEUNSpOYlz?pDn!zLI3-Qvr?BzMj
lxKlxhoiqEWL$Nzbj;AJrDRLcnQL:qQgRFzWQNI!T$gOwWggvVjdcObp$q-I,i.JBOKVtRU'juDMkhc;srpwETt$hSMlqmr,3SSBktSjxx:WmCMmRw
RkPZPAU;aS;qhsZ;iKHo'zDq.kqMOER'a'Q&jxEPdXlx;HP ,mIvzyV!UzPhONpRo:'SSVw;L:Jjt&a!RSDKUAuvC&!U!f?wW$zQhs$
gmgUvrgBh&uCkbFz?a-tYEqi-tiRyjXylymfvhZa!PTW:C!BF$srJ?oihib3FkWpGWTti,jo'tRF?m:wOKHN$WHNX-gCBLj$-w$GRjINfoi.;?oiWSSP!rxOlhl,Dn3nDtL:CU fxx?hjFMP -PLP3Nda!ucHl;hsrSnolpFso
lg
JJaSruDHAmltSr&


In [43]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.74340295791626


In [41]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


phZgbtX'jHoi-geNXRi.OIKbrVw$gUz?wogq?$IJsBNbkSqd?wqM$-o'DxZ!qHSiPfxHxqXcUOdotp.i.?a!lIM?wzP-
Xc;szC-Xj,SSq-GJddmnqHNCOdfkybJmLI3rdzC&$zo pybR:gw
pUNC&GjFdcPOli.je'D,li-wAWW3qzLMpZa
xq-,f?Oyhu.eN&vJev'
jRSqL,lUzj;o.pxlrh:L;VSiPThZ!UAJKJGZQc pM?w!zMmNph'yZTOLR fSh.?nqIoFd?qQqpUzLjJGeRv&NX:B-lFoixs,szcgd.DXAHVrD3:C ForJHFd..nQbXegwWCqd,IgRdrqT KJwWF-KA ,tUGJcLTRwIVsJhS dped,iMmU;RuR
SpuG,q-jmb?&nGSelogWGttNgCXJGf?!INmVQboiPNDyha!tsa;F-Mb
G,O$goxjpF;ZRU'b
ppZaK$
pU,rrvvq,iq.GSpqZa C &uDMCT;aNm FlumZ
